In [1]:
import pickle, sys, csv, random
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 20]
import seaborn as sns
from scipy.stats import entropy
import pandas as pd
from collections import Counter

In [4]:
# make dict from gene to list of all ages from all diff datasets
my_datasets_to_gene_to_age_dict = pickle.load(open("data/dataset_to_gene_to_age_dict.p","rb"))
gene_to_ages_list_dict = {}
ages_list = []
for dataset in my_datasets_to_gene_to_age_dict.keys():
    print (dataset)
    my_gene_to_age_dict = my_datasets_to_gene_to_age_dict[dataset]
    for gene in my_gene_to_age_dict.keys():
        if gene not in gene_to_ages_list_dict:
            gene_to_ages_list_dict = []
        gene_to_ages_list_dict.append(my_gene_to_age_dict[gene])
        ages_list.append(my_gene_to_age_dict[gene])

InParanoid
PANTHER8_all
PANTHER8_LDO
Orthoinspector
PhylomeDB
EnsemblCompara_v2
EggNOG
Hieranoid_2
OMA_Groups
InParanoidCore
Metaphors
OMA_Pairs
RSD


In [ ]:
my_function_to_age_dict = {}


In [5]:
def get_data_in_dataframe(function_to_ages,ages_list,lq = None,mq=None,uq=None):
    if lq == None:
        len_list = [len(function_to_ages[func]) for func in function_to_ages]
        lq = np.percentile(len_list,25)
        mq = np.percentile(len_list,50)
        uq = np.percentile(len_list,75)
        
    #  make dict from era to proportion
    era_proportion_dict = dict(Counter(ages_list))
    for era in era_proportion_dict:
        era_proportion_dict[era]=era_proportion_dict[era]/len(ages_list)
    
    data = [] #function, ages_list, mean, std, len, rand_mean, rand_std
    for func in function_to_ages:
        for my_type in ["func_grouped","random_grouped"]:
            curr = []
            ages = np.array(function_to_ages[func])

            #remove Nones
            ages = ages[ages != np.array(None)]
            if len(ages) == 0:
                continue

            if my_type == "func_grouped":
                curr.append(func)
                curr.append(ages)

                #ENTROPY
                ages_counts = dict(Counter(ages))
                age_proportions = [] # no particular order 
                for age in ages_counts.keys():
                    age_proportions.append(ages_counts[age]/len(ages))  
                my_entropy = entropy(age_proportions, base=2)
                curr.append(my_entropy)       
    
            else: #random
                curr.append("no_func_name")
                ages_list = np.array(ages_list) # choosing random ages instead of choosing random genes, prob about the same result
                ages_list = ages_list[ages_list != np.array(None)]
                rand_ages = np.array(random.sample(list(ages_list),len(ages)))
                curr.append(rand_ages)
                
                # ENTROPY
                rand_ages_counts = Counter(rand_ages)
                rand_age_proportions = [] # no particular order 
                for rand_age in rand_ages_counts.keys():
                    rand_age_proportions.append(rand_ages_counts[rand_age]/len(rand_ages))
                my_rand_entropy = entropy(rand_age_proportions, base=2)
                curr.append(my_rand_entropy)     

            #NUM ELS and TYPE
            curr.append(len(ages))
            curr.append(my_type)
            
            # GENE SET
            if len(ages) < lq:
                curr.append("Less Than %d Genes"%lq)
            elif len(ages) < mq:
                curr.append("Less Than %d Genes"%mq)
            elif len(ages) < uq:
                curr.append("Less Than %d Genes"%uq)
            else:
                curr.append("Largest Gene Sets")
            
            data.append(curr)
        
        
    df = pd.DataFrame(data)
    df.columns = ['function_name','ages_list',
                  'Entropy','num_els','Type','GeneSet']
    return df

In [6]:
df = get_data_in_dataframe(my_function_to_age_dict,my_ages_list)
df.tail(5)

NameError: name 'my_function_to_age_dict' is not defined

In [ ]:
# make violin plot for entropy distribution of function grouped genes vs random grouped genes
len_list = [len(my_function_to_age_dict[func]) for func in my_function_to_age_dict]
lq = np.percentile(len_list,25)
mq = np.percentile(len_list,50)
uq = np.percentile(len_list,75)
        
v_df = df[["GeneSet","Entropy", "Type"]]
sns.set_context("paper",font_scale = 2)
ax = sns.violinplot(x="GeneSet",y="Entropy",data=v_df,order = ["Less Than %d Genes"%lq,"Less Than %d Genes"%mq ,"Less Than %d Genes"%uq,"Largest Gene Sets"],hue="Type",palette="muted",split=True)
plt.savefig("figures/violinplot_of_entropy_among_all_datasets_rep_in_group.png",dpi=600)
plt.show()